In [ ]:
import os
import json
import sys

In [ ]:
# Importamos google drive donde se almacenan los códigos y archivos
from google.colab import drive
drive.mount('/content/drive')

# Código

In [ ]:
# 1. Ruta de trabajo
WORK_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'

# 2. Creación del directorio
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)
    print(f"Directorio creado: {WORK_DIR}")
else:
    print(f"Directorio existente: {WORK_DIR}")

# 3. Cambiar el directorio de trabajo
os.chdir(WORK_DIR)
print(f"Directorio de trabajo actual: {os.getcwd()}")